In [ ]:
import flopy as fp
import geopandas as gp 
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pathlib import Path
import numpy as np

### Set a couple paths to information we will need

In [ ]:
#### Base directory for runs
parent_run_path = Path("../../LPR_pycap_opt/pycap_runs")

#### depletion potential calculations directory
base_run_path = parent_run_path / "pycap_base"


### read in the depletion potetial as calculated by `pycap`

In [ ]:
dp = gp.read_file(base_run_path / 'depletion_potential.json').to_crs(3071)

### now read in the MODFLOW-6 version of the steady-state Little Plover River Model

In [ ]:
sim = fp.mf6.MFSimulation.load(sim_ws = '../steady_state_mf6')
m = sim.get_model()

### We will need a `modelgrid` object to locate wells and gages on the MODFLOW grid

In [ ]:
modelgrid = fp.discretization.StructuredGrid(xoff=1795400*.3048,  
                                             yoff=1424400*.3048,
                                             botm=m.dis.botm.array,
                                             delr = m.dis.delr.array*.3048,
                                             delc = m.dis.delc.array*.3048,
                                                
                                             crs=3071)


### Let's write out the grid as a shapefile and then explore the model 

In [ ]:
modelgrid.write_shapefile('grid.shp')
grid_shp = gp.read_file('grid.shp').set_crs(3071)
mf6outline = grid_shp.dissolve()

In [ ]:
mmap=mf6outline.explore()
dp.explore(column="Depletion_Potential",
                  vmin=0,vmax=1,m=mmap,
                  style_kwds={"style_function":
                                  lambda x: 
                                  {"radius":x["properties"]["Depletion_Potential"]*15}})


## let's choose a well to compare between MODFLOW and pycap for depletion potential

In [ ]:
eval_well = '23627'

In [ ]:
wellrc = gp.sjoin(dp.loc[dp.name==eval_well],
         grid_shp,
         how='inner',
         predicate='intersects')

In [ ]:
wells = pd.DataFrame.from_records(m.wel.stress_period_data.array[0])
wells['r'] = [i[1] for i in wells.cellid]
wells['c'] = [i[2] for i in wells.cellid]
wells

In [ ]:
wells.loc[(wells.r==wellrc.iloc[0].row-1) & (wells.c==wellrc.iloc[0].column-1)]

In [ ]:
wellrc

In [ ]:
hoover_gage_loc = gp.GeoDataFrame(index=['hoover'],geometry=[Point(557498.46,444568.09)],crs=3071)

In [ ]:
gage_rc = wellrc = gp.sjoin(hoover_gage_loc,
         grid_shp,
         how='inner',
         predicate='intersects')

In [ ]:
gage_rc

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
pmv = fp.plot.PlotMapView(m, ax=ax)
# lc = pmv.plot_grid(lw=0.5)
pmv.plot_bc("WEL", plotAll=True)
pmv.plot_bc("SFR", plotAll=True)
pmv.plot_bc("CHD", plotAll=True)
ax.set_xlabel(f'{modelgrid.units.capitalize()} easting, {modelgrid.crs.name}')
ax.set_ylabel(f'{modelgrid.units.capitalize()} northing, {modelgrid.crs.name}')

In [ ]:
sfr_lox = pd.DataFrame.from_records(m.sfr.packagedata.array)
sfr_lox['r'] = [i[1] for i in sfr_lox.cellid]
sfr_lox['c'] = [i[2] for i in sfr_lox.cellid]
hoover_reach = sfr_lox.loc[(sfr_lox.r==gage_rc.iloc[0].row-1)& (sfr_lox.c==gage_rc.iloc[0].column-1)].ifno.values[0]

In [ ]:
hoover_reach

### we can make a quick obsevation file for SFR following guidance here:
https://modflow6.readthedocs.io/en/stable/_mf6io/gwf-sfr.html

In [ ]:
with open('../steady_state_mf6/sfr_obs.obs', 'w') as ofp:
    ofp.write('BEGIN OPTIONS\n\tDIGITS 8\n\tPRINT_INPUT\nEND OPTIONS\n\n')
    ofp.write('BEGIN CONTINUOUS FILEOUT hoover.sfr.csv\n')
    ofp.write(f'\thooverflow DOWNSTREAM-FLOW {hoover_reach}\n')
    ofp.write('END CONTINUOUS')

### we also have to update the SFR file to recognize the observation file invocation

In [ ]:
# read in the existing SFR file
sfr_old = [i.rstrip() for i in open('../steady_state_mf6/lpr_ss.sfr.orig','r').readlines()]

In [ ]:
with open('../steady_state_mf6/lpr_ss.sfr','w') as ofp:
    for line in sfr_old:
        if 'begin' in line.lower() and 'options' in line.lower():
            ofp.write('BEGIN Options\n\tOBS6 FILEIN sfr_obs.obs\n')
        else:
            ofp.write(line + '\n')

### now if we run MF6 again, we should generate an observation of flow at hoover

In [ ]:
hoover_q = pd.read_csv('../steady_state_mf6/hoover.sfr.csv')
hoover_q['HOOVERFLOW']

### this is in CFD (cubic feet per day) so need to convert to CFS

In [ ]:
hoover_q['HOOVERFLOW']/24/60/60

## Quick bespoke DP calculation

In [ ]:
hoover_q_inc = pd.read_csv('../steady_state_mf6_DP/hoover.sfr.csv')
hoover_q_inc['HOOVERFLOW']/24/60/60

In [ ]:
dQ_well = (6.15207861E+03-4.15207861E+03)

dQ_well

In [ ]:
(dQ_well)/(np.abs(hoover_q['HOOVERFLOW'])-np.abs(hoover_q_inc['HOOVERFLOW']))

In [ ]:
dp.loc[dp.name==eval_well]